<a href="https://www.kaggle.com/code/fbasampaio/naive-bayes-fraud-prevention?scriptVersionId=220847343" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_id = pd.read_csv("/kaggle/input/ieee-fraud-detection/train_identity.csv")
df_trans = pd.read_csv("/kaggle/input/ieee-fraud-detection/train_transaction.csv")
df_trans.head(4)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
train=df_trans.merge(df_id,how='left',left_index=True,right_index=True)

del df_trans,df_id

train.head(2)

,TransactionID_x,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device


In [4]:
 # Replace null values with the mode in all columns
train = train.apply(lambda col: col.fillna(col.mode()[0] if not col.mode().empty else col), axis=0)
train.head()

,TransactionID_x,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,321.0,150.0,discover,142.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,chrome 62.0,24.0,1920x1080,match_status:2,F,F,T,T,desktop,Windows
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,chrome 62.0,24.0,1920x1080,match_status:2,F,F,T,T,desktop,Windows
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,chrome 62.0,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS


In [5]:
from sklearn.preprocessing import LabelEncoder

# Instanciar o LabelEncoder
label_encoder = LabelEncoder()

# Aplicar Label Encoding em todas as colunas categóricas
for col in train.columns:
    if train[col].dtype == 'object':  # Verifica se a coluna é do tipo 'object'
        train[col] = label_encoder.fit_transform(train[col])

# Exibir o DataFrame resultante
train.head()

,TransactionID_x,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,4,13926,321.0,150.0,1,142.0,...,123,32.0,164,3,1,0,1,1,1,954
1,2987001,0,86401,29.0,4,2755,404.0,150.0,2,102.0,...,98,32.0,48,2,1,0,0,1,1,1727
2,2987002,0,86469,59.0,4,4663,490.0,150.0,3,166.0,...,44,24.0,132,3,0,0,1,1,0,1598
3,2987003,0,86499,50.0,4,18132,567.0,150.0,2,117.0,...,44,24.0,132,3,0,0,1,1,0,1598
4,2987004,0,86506,50.0,1,4497,514.0,150.0,2,102.0,...,44,24.0,40,3,1,0,1,1,0,723


In [6]:
from sklearn.model_selection import train_test_split

# Separate the features (X) and the target variable (y)
X = train.drop('isFraud', axis=1)  # Remove the 'isFraud' column from X
y = train['isFraud']  # y contains only the 'isFraud' column

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [7]:
from sklearn.model_selection import train_test_split  # Importando train_test_split para dividir o conjunto de dados
from sklearn.naive_bayes import GaussianNB  # Importando GaussianNB para criar o modelo Naive Bayes
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score  # Importando várias métricas para avaliação do modelo


# Criando o modelo Naive Bayes
model = GaussianNB()

# Treinando o modelo com os dados balanceados de treinamento
model.fit(X_train, y_train)  # Certifique-se de que esta linha seja executada antes de fazer previsões

# Fazendo previsões no conjunto de teste
y_pred = model.predict(X_test)  # Previsões das classes
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probabilidades para a classe positiva

# Avaliando o modelo
accuracy = accuracy_score(y_test, y_pred)  # Cálculo da acurácia
precision = precision_score(y_test, y_pred)  # Cálculo da precisão
recall = recall_score(y_test, y_pred)  # Cálculo do recall
f1 = f1_score(y_test, y_pred)  # Cálculo do F1-Score
roc_auc = roc_auc_score(y_test, y_pred_proba)  # Cálculo do AUC-ROC
conf_matrix = confusion_matrix(y_test, y_pred)  # Cálculo da matriz de confusão

# Exibindo as métricas de avaliação
print(f"Accuracy: {accuracy:.2f}")  # Exibindo a acurácia
print(f"Precision: {precision:.2f}")  # Exibindo a precisão
print(f"Recall: {recall:.2f}")  # Exibindo o recall
print(f"F1-Score: {f1:.2f}")  # Exibindo o F1-Score
print(f"AUC-ROC: {roc_auc:.2f}")  # Exibindo o AUC-ROC
print("Confusion Matrix:")  # Exibindo a matriz de confusão
print(conf_matrix)  # Imprimindo a matriz de confusão


Accuracy: 0.06
Precision: 0.03
Recall: 0.96
F1-Score: 0.07
AUC-ROC: 0.48
Confusion Matrix:
[[  3303 110671]
 [   179   3955]]
